In [1]:
from tensorflow.keras.optimizers import Adam
from Models.Loss import NLL
from Models.Distributions import *
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential, Model
from Utils.Dataset import getData
from Utils.transform import cutOut,LinBin,Normalize,NormalizePerImage
from tensorflow.keras.callbacks import *
from Models.Utils import *
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
import os
import cv2 as cv
import numpy as np

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)


Num GPUs: 1


In [2]:
def five_class_categorical(input_shape,
                          activation_hidden="relu"):


    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(20, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(25, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(25, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(25, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32

    
    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64

    
    layer = Conv2D(1, (7, 7), activation="relu")(up01)  # 1 x 64x64


    prob = Flatten()(layer)
    
    prob      = Dense(256)(prob)
    

    prob = Dense(50*50*5,activation="linear")(prob)
    
    prob = tf.keras.layers.Reshape((50,50,1,5))(prob)

    
    prob = tf.math.softmax(prob,axis=-1)
    #prob = prob)
    #input_dist= tf.concat([cat,count,prob],axis=-1)
    
    output_dist = tfp.layers.DistributionLambda(
        name="DistributionLayer",
        make_distribution_fn=lambda t: tfp.distributions.Independent(
        tfd.Categorical(logits=tf.math.log(t[...,:,:,:]))
        ,name="categorical",reinterpreted_batch_ndims=0 ))
    
    #output_dist = tfp.layers.DistributionLambda(tfd.Categorical,name="categorical"))
    
    

    output = output_dist(prob)
    #output = tf.math.softmax(prob,axis=-1)
    model = Model(inputs=inputs, outputs=output)
    return model

In [5]:
BATCH_SIZE = 100
DIMENSION = (64,64)
CHANNELS = 5
MODELPATH = "./Models_weights"
MODELNAME = "five_class_categorical"



def getModel():
    modelpath = MODELPATH
    modelname = MODELNAME

    
    if not os.path.exists(modelpath):
            os.mkdir(modelpath)

    modelpath = os.path.join(modelpath,modelname)

    if not os.path.exists(modelpath):
        os.mkdir(modelpath)

    
    input_shape = (*DIMENSION,CHANNELS)

    model = five_class_categorical(
                input_shape=input_shape
                )

    y_transform = [cutOut([7,57,7,57]),LinBin(56)]
    x_transform = [Normalize(0.007742631458799244, 0.015872015890555563)]
    #x_transform = [NormalizePerImage()]

    
    train,test = getData(BATCH_SIZE,
                         DIMENSION,CHANNELS,
                         timeToPred=10,
                         y_transform=y_transform,
                         x_transform=x_transform)
    
    
    def NLL(y_true, y_hat):
        return -y_hat.log_prob(y_true)

    #neg_log_likelihood = lambda x, rv_x: tf.math.reduce_mean(-rv_x.log_prob(x))
    
    model.compile(loss=NLL,
                  optimizer=Adam( lr= 1e-2 ))
    model.summary()
    modelpath_h5 = os.path.join(modelpath,
                            modelname+'-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5')

    checkpoint = ModelCheckpoint(modelpath_h5,
                                 verbose=0,
                                 monitor='val_loss',
                                 save_best_only=True,
                                 mode='auto')

    return model,checkpoint,modelpath,train,test

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [6]:

modelpath = MODELPATH
modelname = MODELNAME

model,checkpoint,modelpath,train,test = getModel()


history_path = os.path.join(modelpath,modelname+"_history")
laststate = getBestState(modelpath,history_path)
test.setWiggle_off()
train.setWiggle_off()

print(model(train[0][0]))
#print(model(train[0][0]).prob(train[0][1]))

if laststate:
    epoch = laststate["epoch"]
    model.load_weights(laststate["modelpath"])


    loss = model.evaluate(x=test, verbose=2)
    print("Restored model, loss: {:5.5f}".format(loss))

    history = model.fit(train,
                        validation_data = test,
                        shuffle         = True,
                        epochs          = 100+epoch,
                        initial_epoch   = epoch,
                        batch_size      = BATCH_SIZE,
                        callbacks       = checkpoint)

    history = mergeHist(laststate["history"],history.history)

else:
    history = model.fit(train,
                        validation_data = test,
                        shuffle         = True,
                        epochs          = 100,
                        batch_size      = BATCH_SIZE,
                        callbacks       = checkpoint)

    history = history.history





saveHistory(history_path,history)
plotHistory(history,history_path,title="five_class_categoricall NLL-loss")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 5)]  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 64, 64, 20)   920         input_2[0][0]                    
__________________________________________________________________________________________________
activation_4 (Activation)       (None, 64, 64, 20)   0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 32, 32, 20)   0           activation_4[0][0]               
____________________________________________________________________________________________

KeyboardInterrupt: 